In [ ]:
!pip list

In [ ]:
!pip install huggingface_hub transformers accelerate

In [4]:
import torch
import requests
from PIL import Image
from transformers import AutoModelForCausalLM


In [21]:
MODEL_PATH = "AIDC-AI/Ovis2.5-2B"

# Thinking mode & budget
enable_thinking = False
enable_thinking_budget = False  # Only effective if enable_thinking is True.

# Total tokens for thinking + answer. Ensure: max_new_tokens > thinking_budget + 25
max_new_tokens = 3072
thinking_budget = 2048

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
).cuda()

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is 542C-8606

 Directory of C:\Users\User\Desktop\OpenCV University\vlm-bench\hface\ovis

16-09-2025  18:07    <DIR>          .
16-09-2025  17:40    <DIR>          ..
16-09-2025  17:41    <DIR>          .ipynb_checkpoints
09-09-2025  16:02            88,258 bird.jpg
16-09-2025  18:07            16,307 Ovis2.5-2B.ipynb
               2 File(s)        104,565 bytes
               3 Dir(s)  170,820,112,384 bytes free


In [19]:
img = 'bird.jpg'

In [23]:
messages = [{
    "role": "user",
    "content": [
        {"type": "image", "image": Image.open(img)},
        {"type": "text", "text": "Describe the image in 100 words"},
    ],
}]

input_ids, pixel_values, grid_thws = model.preprocess_inputs(
    messages=messages,
    add_generation_prompt=True,
    enable_thinking=enable_thinking
)
input_ids = input_ids.cuda()
pixel_values = pixel_values.cuda() if pixel_values is not None else None
grid_thws = grid_thws.cuda() if grid_thws is not None else None

outputs = model.generate(
    inputs=input_ids,
    pixel_values=pixel_values,
    grid_thws=grid_thws,
    enable_thinking=enable_thinking,
    enable_thinking_budget=enable_thinking_budget,
    max_new_tokens=max_new_tokens,
    thinking_budget=thinking_budget,
)

response = model.text_tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


A hummingbird hovers near a vibrant flower, feeding on nectar. The bird's iridescent green feathers and long beak are visible as it hovers. The flower features bright orange and yellow petals, with some petals appearing dried. The background is softly blurred, creating a warm, yellowish hue that highlights the bird and flower. Another similar flower is partially visible on the left.
